In [1]:
import numpy as np
from numpy.core.fromnumeric import argmin
import functions as func
import plots as mypt
import time

def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)
'''
def trust_region_subproblem(gradient, hessian, lambda_0, delta, max_iter=1000):
    d = hessian.shape[0]
    lambda_j, vector = np.linalg.eig(hessian)
    lambda_1 = min(lambda_j)
    min_index = argmin(lambda_j)
    lam = lambda_0
    p = np.linalg.inv(hessian) @ (-gradient) #initial p
    count = 0
    #print(hessian)
    while lam >= -lambda_1:
        count += 1
        B_ = hessian + lam * np.identity(d)
        # try:
        if vector[:, min_index].T @ gradient == 0:
            p = hard_case(gradient, hessian, delta, lambda_j, vector, min_index)
            return p
        else:
            p = np.linalg.inv(B_) @ (-gradient)
        R = np.linalg.cholesky(B_)
        q = np.linalg.inv(R.T) @ p

        lam_new = lam + (np.linalg.norm(p) / np.linalg.norm(q)) ** 2 * (
            (np.linalg.norm(p) - delta) / delta
        )
        #print(lam)
        if lam_new == lam or (count >= max_iter and np.linalg.norm(p) <= delta) or (lam_new < -lambda_1):
            # return p
            break
        else:
            lam = lam_new
    print(count)
    return p
'''
def sub_solve_2(gx, hx, delta):
    atol = 1e-8
    gx_norm = np.linalg.norm(gx)
    lmds, Q = np.linalg.eig(hx)
    minIdx = np.argmin(lmds)
    #is_psd  = lmds[minIdx] >= -atol
    if Q[:, minIdx].T @ gx != 0:
        if lmds[minIdx] > atol:
            lmd = 0
        else:
            lmd = -lmds[minIdx] + atol
        while True:
            #B_inv = Q @ np.diag(1 / (lmds + lmd)) @ Q.T
            B_inv = Q @ np.diag(1 / (lmds + lmd + atol)) @ Q.T
            p     = B_inv @ -gx
            p_n   = np.linalg.norm(p)
            if p_n <= delta + atol:
                break
            p_n2  = p_n*p_n
            q_n2  = p.T @ B_inv @ p
            lmd   = lmd + (p_n2/q_n2) * (p_n - delta) / delta
    else:
        #print(f"Hard case! (||gx|| = {gx_norm})")
        p_n = delta
        if gx_norm == 0:
            p = delta*Q[:, minIdx]
        else:
            p = delta*(-gx/gx_norm)
            gBg = gx.T @ hx @ gx
            if gBg > 0:
                tau = (gx_norm*gx_norm*gx_norm) / (delta * gBg)
                if tau < 1:
                    p, p_n = p*tau, p_n*tau
    return p
'''
def hard_case(gradient, hessian, delta, lambda_j, vector, min_index):
    temp = gradient.T @ hessian @ gradient
    tau = min(1, np.linalg.norm(gradient) ** 3 / (delta * temp))
    z = vector[:, min_index]  # eigenvector of B corresponding to lamda_1
    lamda_1 = lambda_j[min_index]
    p = 0
    for i in range(np.shape(lambda_j)[0]):
        if lamda_1 != lambda_j[i]:
            p += -((vector[:, i].T @ gradient) / (lambda_j[i] - lamda_1) * vector[:, i])
    p += tau * z
    return p

def cauchy_point(gradient, hessian, delta):
    temp = gradient.T @ hessian @ gradient
    if temp <= 0:
        tau = 1
    else:
        tau = min(1, np.linalg.norm(gradient) ** 3 / (delta * temp))
    return -tau * delta / np.linalg.norm(gradient) * gradient
'''
def trust_region(_x, fun, fun_g, fun_h, _delta, delta_max, eta=1e-4, max_iter=1000, tol=1e-6):
	xs = [_x] #store x_0
	deltas = [_delta] #store delta_0
	#Bs = [np.identity(len(_x))] #store B_0
	x = _x #initialize x_k to x_0
	delta = _delta #initialize delta_k to delta_0
	B = np.identity(len(_x)) #initialize B_k as identity matrix B_0
	count = 0 #store iterations number
	while np.linalg.norm(fun_g(x))>tol: #until reach the tolerance
		if(count >= max_iter): break #until reach the maximum iterations
		count += 1
		g = fun_g(x)
		#B = fun_h(x) #this is same as last time in assignment 4
		s = sub_solve_2(g, B, delta)
		'''
		if is_pos_def(B):
			if np.linalg.norm(np.linalg.inv(B) @ g) <= delta:
				s = -np.linalg.inv(B) @ g
			else:
				s = trust_region_subproblem(g, B, 0, delta)
		else:
			s = trust_region_subproblem(g, B, -min(np.linalg.eigvals(B)+1e-8), delta)  #set the first lambda to almost -lambda_1
		'''
		y = fun_g(x+s) - g
		ared = fun(x) - fun(x+s) #actual reduction
		pred = -(g.T @ s + (s.T @ B @ s)/2) #predicted reduction
		rho = ared / pred
		#print("ared:",ared,"\tpred:",pred,"\tdelta:",delta,"\t||g||",np.linalg.norm(fun_g(x)))
		if rho > eta:
			x = x + s
		else:
			pass #x_k+1=x_k
		if rho > 0.75:
			if np.linalg.norm(s) <= 0.8*delta:
				pass
			else:
				delta=2*delta
		elif 0.1 <= rho <= 0.75:
			pass
		else:
			delta=0.5*delta
		r=1e-8; tmp=y-B@s
		if np.array_equal(y, B @ s):
			pass
		elif np.abs(s.T @ tmp) < 1e-8:
			pass
		elif np.abs(s.T @ tmp)>=r*np.linalg.norm(s)*np.linalg.norm(tmp): #6.26
			B=B+np.outer(tmp,tmp)/(tmp.T @ s) #6.24, SR1
		else:
			pass
		xs.append(x)
		deltas.append(delta)
	return np.array(xs), fun(x), count, np.array(deltas)

def test(fun, fun_g, fun_h, dim=2, name="",tol=1e-6):
    time_s,time_f = 0,0
    iter_s,iter_f = 0,0
    succ_s = 0
    val_s,val_f = 0,0
    x_s,x_f = 0,0
    points = 100
    for i in range(points):
        #x = np.random.rand(length) * 5
        x = np.zeros(dim)
        for j in range(dim):
            x[j] = np.random.uniform(-5, 5)
        tmp1 = time.process_time()
        xs, val, c, dels = trust_region(x, fun, fun_g, fun_h,0.1,10,tol=tol)
        tmp2 = time.process_time()
        x_ = xs[-1]
        if np.linalg.norm(fun_g(x_)) <= tol:
            succ_s += 1
            time_s += tmp2-tmp1
            iter_s += c
            #x_s += x_
            val_s += val
        else:
            time_f += tmp2-tmp1
            iter_f += c
            #x_f += x_
            val_f += val
    succ_f=points-succ_s
    '''
    if(succ_s>0 and succ_f>0):
        print(name+" succ", "\ttime:", round(time_s/succ_s,6), "\titer:", round(iter_s/succ_s,4), "\ty:", val_s/succ_s, "\tx:", x_s/succ_s, "\tsucc rate:", succ_s/points)
        print(name+" fail", "\ttime:", round(time_f/succ_f,6), "\titer:", round(iter_f/succ_f,4), "\ty:", val_f/succ_f, "\tx:", x_f/succ_f, "\tfail rate:", succ_f/points)
    elif(succ_s==0):
        #print(name+" without successful convergence.")
        print(name+" fail", "\ttime:", round(time_f/succ_f,6), "\titer:", round(iter_f/succ_f,4), "\ty:", val_f/succ_f, "\tx:", x_f/succ_f, "\tfail rate:", succ_f/points)
    elif(succ_f==0):
        print(name+" succ", "\ttime:", round(time_s/succ_s,6), "\titer:", round(iter_s/succ_s,4), "\ty:", val_s/succ_s, "\tx:", x_s/succ_s, "\tsucc rate:", succ_s/points)
        #print(name+" without failed convergence.")
    '''
    if(succ_s>0 and succ_f>0):
        print("& succ", "&", round(time_s/succ_s,6), "&", round(iter_s/succ_s,4), "&", succ_s/points)
        print("& fail", "&", round(time_f/succ_f,6), "&", round(iter_f/succ_f,4), "&", succ_f/points)
    elif(succ_s==0):
        #print(name+" without successful convergence.")
        print("& fail", "&", round(time_f/succ_f,6), "&", round(iter_f/succ_f,4), "&", succ_f/points)
    elif(succ_f==0):
        print("& succ", "&", round(time_s/succ_s,6), "&", round(iter_s/succ_s,4), "&", succ_s/points)
        #print(name+" without failed convergence.")

In [2]:
'''
x0 = np.array([-1.8, 2.1])

X1,X2,Y=mypt.gen2d(0.01,-3.5,3.5,2) #generate points set for the contour plot

f1_xs, f1_y, f1_count, f1_dels = trust_region(np.array(x0), func.f1, func.g1, func.h1, 0.1, 10)
f2_xs, f2_y, f2_count, f2_dels = trust_region(np.array(x0), func.f2, func.g2, func.h2, 0.1, 10)
f3_xs, f3_y, f3_count, f3_dels = trust_region(np.array(x0), func.f3, func.g3, func.h3, 0.1, 10)
f4_xs, f4_y, f4_count, f4_dels = trust_region(np.array(x0), func.f4, func.g4, func.h4, 0.1, 10)
f5_xs, f5_y, f5_count, f5_dels = trust_region(np.array(x0), func.f4, func.g4, func.h4, 0.1, 10)

mypt.plot_deltas(f1_dels, name="f1")
mypt.plot_deltas(f2_dels, name="f2")
mypt.plot_deltas(f3_dels, name="f3")
mypt.plot_deltas(f4_dels, name="f4")
mypt.plot_deltas(f5_dels, name="f5")
'''

'\nx0 = np.array([-1.8, 2.1])\n\nX1,X2,Y=mypt.gen2d(0.01,-3.5,3.5,2) #generate points set for the contour plot\n\nf1_xs, f1_y, f1_count, f1_dels = trust_region(np.array(x0), func.f1, func.g1, func.h1, 0.1, 10)\nf2_xs, f2_y, f2_count, f2_dels = trust_region(np.array(x0), func.f2, func.g2, func.h2, 0.1, 10)\nf3_xs, f3_y, f3_count, f3_dels = trust_region(np.array(x0), func.f3, func.g3, func.h3, 0.1, 10)\nf4_xs, f4_y, f4_count, f4_dels = trust_region(np.array(x0), func.f4, func.g4, func.h4, 0.1, 10)\nf5_xs, f5_y, f5_count, f5_dels = trust_region(np.array(x0), func.f4, func.g4, func.h4, 0.1, 10)\n\nmypt.plot_deltas(f1_dels, name="f1")\nmypt.plot_deltas(f2_dels, name="f2")\nmypt.plot_deltas(f3_dels, name="f3")\nmypt.plot_deltas(f4_dels, name="f4")\nmypt.plot_deltas(f5_dels, name="f5")\n'

In [3]:
'''
mypt.plot_convergence(f1_xs,func.f1,name="f1")
mypt.plot_convergence(f2_xs,func.f2,name="f2")
mypt.plot_convergence(f3_xs,func.f3,name="f3")
mypt.plot_convergence(f4_xs,func.f4,name="f4")
mypt.plot_convergence(f5_xs,func.f5,name="f5")
'''

'\nmypt.plot_convergence(f1_xs,func.f1,name="f1")\nmypt.plot_convergence(f2_xs,func.f2,name="f2")\nmypt.plot_convergence(f3_xs,func.f3,name="f3")\nmypt.plot_convergence(f4_xs,func.f4,name="f4")\nmypt.plot_convergence(f5_xs,func.f5,name="f5")\n'

In [4]:
'''
mypt.plot2d(X1,X2,Y,x0,func.f1,f1_xs,f1_count,f1_y,name="f1")
mypt.plot2d(X1,X2,Y,x0,func.f2,f2_xs,f2_count,f2_y,name="f2")
mypt.plot2d(X1,X2,Y,x0,func.f3,f3_xs,f3_count,f3_y,name="f3")
mypt.plot2d(X1,X2,Y,x0,func.f4,f4_xs,f4_count,f4_y,name="f4")
mypt.plot2d(X1,X2,Y,x0,func.f5,f5_xs,f5_count,f5_y,name="f5")
'''

'\nmypt.plot2d(X1,X2,Y,x0,func.f1,f1_xs,f1_count,f1_y,name="f1")\nmypt.plot2d(X1,X2,Y,x0,func.f2,f2_xs,f2_count,f2_y,name="f2")\nmypt.plot2d(X1,X2,Y,x0,func.f3,f3_xs,f3_count,f3_y,name="f3")\nmypt.plot2d(X1,X2,Y,x0,func.f4,f4_xs,f4_count,f4_y,name="f4")\nmypt.plot2d(X1,X2,Y,x0,func.f5,f5_xs,f5_count,f5_y,name="f5")\n'

In [5]:
'''
test(func.f1, func.g1, func.h1, name="f1")
test(func.f2, func.g2, func.h2, name="f2")
test(func.f3, func.g3, func.h3, name="f3")
test(func.f4, func.g4, func.h4, name="f4")
test(func.f5, func.g5, func.h5, name="f5")
'''

'\ntest(func.f1, func.g1, func.h1, name="f1")\ntest(func.f2, func.g2, func.h2, name="f2")\ntest(func.f3, func.g3, func.h3, name="f3")\ntest(func.f4, func.g4, func.h4, name="f4")\ntest(func.f5, func.g5, func.h5, name="f5")\n'

In [22]:
x_test = np.array([1,-1.6,0.9,1.2]) #4d starting point
dim = x_test.shape[0]
x_min = np.zeros(dim) #4d minimum point, with result is 0
#print(x_test,x_min)

Shape = np.identity(dim)

for i in range(7):
    
    for j in range(dim):
        for k in range(dim):
            if j==k:
                Shape[j,k] = Shape[j,k]+i*(j*k)**(3*i) #Shape transforms by changing eigenvalues
    
    #dim=i
    #Shape = np.identity(dim)
    #print(Shape)
    #Shape[dim-1,dim-1] = 10
    #print(Shape)
    print('(',np.linalg.eig(Shape)[0][0],np.linalg.eig(Shape)[0][1],np.linalg.eig(Shape)[0][2],np.linalg.eig(Shape)[0][3],') ',end='')
    f6 = lambda x: x.T @ Shape @ x #ellipsoid function, min when all x=0
    g6 = lambda x: Shape @ x + Shape.T @ x
    h6 = 0 #don't need
    #print('minimum:',f6(x_min))
    #print(dim,'',end='')
    test(f6, g6, h6, dim, name="f6") #test on different dimensions
    '''
    for j in range(100):
        tmp1 = time.process_time()
        xs, val, c, dels = trust_region(x_test, f6, g6, h6,0.1,10,tol=1e-6)
        tmp2 = time.process_time()
        x_ = xs[-1]
    #print(i+1,end='')
    if np.linalg.norm(g6(x_)) <= 1e-6:
        print("\tsucc.\t",end='')
    print('minimize:',val,'\ttime:',tmp2-tmp1,'\titerations:',c)
    '''

( 1.0 1.0 1.0 1.0 ) & succ & 0.001563 & 6.35 & 1.0
( 1.0 2.0 65.0 730.0 ) & succ & 0.002656 & 6.99 & 1.0
( 1.0 4.0 8257.0 1063612.0 ) & succ & 0.002344 & 7.98 & 1.0
( 1.0 7.0 794689.0 1163325079.0 ) & succ & 0.002656 & 8.24 & 1.0
( 1.0 11.0 67903553.0 1130881471003.0 ) & succ & 0.002656 & 9.21 & 1.0
( 1.0 16.0 5436612673.0 1030586541944248.0 ) & succ & 0.002188 & 9.78 & 1.0
( 1.0 22.0 417753473089.0 9.01598398323939e+17 ) & succ & 0.004577 & 25.8485 & 0.99
& fail & 0.125 & 1000.0 & 0.01
